In [0]:
# read raw csv

df_raw= spark.read.option("header",True).csv("/databricks-datasets/retail-org/customers/")

#save as delta table(bronze layer)

df_raw.write.format("delta").option("mergeSchema","true").mode("overwrite").saveAsTable("bronze_customers")

In [0]:
from pyspark.sql.functions import col,lower,trim

#read from bronze table

df_bronze= spark.read.table("bronze_customers")

#clean

df_silver=df_bronze.withColumn("customer_name", lower(trim(col("customer_name")))).dropna(subset=["customer_id","customer_name"])

# save as silver table
df_silver.write.format("delta").option("mergeSchema",'true').mode("overwrite").saveAsTable("silver_customers")

display(df_silver)

In [0]:
# read from silver table

df_silver=spark.read.table('silver_customers')

#aggregate

df_gold=df_silver.groupBy("state").count()

#save as gold table

df_gold.write.format("delta").option("mergeSchema",'true').mode("overwrite").saveAsTable("customer_summary")

In [0]:
%sql
select * from customer_summary

In [0]:
%sql
describe history customer_summary